<font color='#3498DB'><center><h2>Utilizing Transformer Representations Efficiently</h2>- Strategies for Utilizing and Deepening Transformer Contextual Representations</center></font>
<br>
<center></center>

<font color='#3498DB'><h3>Introduction</h3></font>
We are accustomed to the canonical way of fine-tuning: append just an additional output layer after Transformer for downstream tasks or back-end part of models which takes representations from the last layer of the pre-trained language models as the default input. 

However, due to the multi-layer structure of Transformers, different layers capture different levels of representations. They learn a rich hierarchy of linguistic information i.e. with surface features in lower layers, syntactic features in middle layers, and semantic features in higher layers.

![transformer_intermediate_representation](http://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png)
<br>

The BERT authors tested word-embedding strategies by feeding different vector combinations as input features to a BiLSTM used on a named entity recognition task and observing the resulting F1 scores. Concatenation of the last four layers produced the best results.

This is partially demonstrated by noting that the different layers of BERT encode very different kinds of information, so the appropriate pooling strategy will change depending on the application because different layers encode different kinds of information. This holds true for other variants as well.

<font color='#3498DB'><h3>Overview</h3></font>
<br>
In HuggingFace Transformers there are 2 main outputs and 3 if configured; that we receive after giving input_ids and attention_mask as input.

 - **pooler output** (batch size, hidden size) - Last layer hidden-state of the first token of the sequence
 - **last hidden state** (batch size, seq Len, hidden size) which is the sequence of hidden states at the output of the last layer.
 - **hidden states** (n layers, batch size, seq Len, hidden size) - Hidden states for all layers and for all ids.

Below, we can visualize Embeddings from 5-Fold finetuned RoBERTa [here](https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-infer) and for all competition data points. The continuous targets are split into bins.

*Note 1: I have included the script to generate below plot in the dataset.*  
*Note 2: I have used `torch.no_grad` to fetch outputs from transformer in each technique since gradients gets accumulated and that results in OOM issue. Don't use this during training.*

In [ ]:
from IPython.display import Image
Image("../input/visualizemodelembeddings/Pooler Embedding for 5-Fold RoBERTa Model.jpg", height=500)

We can see that the model can differentiate between the training data points pretty good just by using pooler embeddings but we still have few hard samples. For this, utilizing intermediate representations from various layers will provide better representations as it will help in incorporating more information. 

The notebook will show many different ways these outputs and hidden representations can be utilized to do much more than just adding an output layer. Below are the various techniques we will be implementing.

<font color='#3498DB'><h3>Contents</h3></font>
- [**Pooler Output**](#section1)
- [**Last Hidden State Output**](#section2)
  - [CLS Embeddings](#None)
    - [Introduction](#None)
    - [Implementation](#None)
  - [Mean Pooling](#None)
    - [Introduction](#None)
    - [Implementation](#None)
  - [Max Pooling](#None)
    - [Introduction](#None)
    - [Implementation](#None)
  - [Mean + Max Pooling](#None)
    - [Introduction](#None)
    - [Implementation](#None)
  - [Conv1D Pooling](#None)
    - [Introduction](#None)
    - [Implementation](#None)
- [**Hidden Layers Output**](#section3)
  - [Layerwise CLS Embeddings](#None)
    - [Introduction](#None)
    - [Implementation](#None)
  - [Concatenate Pooling](#None)
    - [Introduction](#None)
    - [Implementation](#None)
  - [Weighted Layer Pooling](#None)
    - [Introduction](#None)
    - [Implementation](#None)
  - [LSTM / GRU Pooling](#None)
    - [Introduction](#None)
    - [Implementation](#None)
  - [Attention Pooling](#None)
    - [Introduction](#None)
    - [Implementation](#None)
  - [WKPooling](#None)
    - [Introduction](#None)
    - [Implementation](#None)
- [**Review**](#None)
- [**Case Study**](#None)
- [**References and Resources**](#None)
- [**Ending Notes**](#None)

<font color='#3498DB'><h3>What's New?</h3></font>
1. [SWA, Apex AMP & Interpreting Transformers in Torch](https://www.kaggle.com/rhtsingh/swa-apex-amp-interpreting-transformers-in-torch) notebook is an implementation of the Stochastic Weight Averaging technique with NVIDIA Apex on transformers using PyTorch. The notebook also implements how to interactively interpret Transformers using LIT (Language Interpretability Tool) a platform for NLP model understanding.   
It has in-depth explanations and code implementations for,
 - SWA 
 - Apex AMP
 - Weighted Layer Pooling
 - MADGRAD Optimizer
 - Grouped LLRD
 - Language Interpretibility Tool
    - Attention Visualization
    - Saliency Maps
    - Integrated Gradients
    - LIME 
    - Embedding Space (UMAP & PCA)
    - Counterfactual generation
    - And many more ...

2. [On Stability of Few-Sample Transformer Fine-Tuning](https://www.kaggle.com/rhtsingh/on-stability-of-few-sample-transformer-fine-tuning) notebook goes over various remedies to increase few-sample fine-tuning stability and they show a significant performance improvement over simple finetuning methods. The methods explained in the notebook are - 
 - Debiasing Omission In BertADAM
 - Re-Initializing Transformer Layers
 - Utilizing Intermediate Layers
 - Layer-wise Learning Rate Decay (LLRD) 
 - Mixout Regularization
 - Pre-trained Weight Decay
 - Stochastic Weight Averaging. 
 
3. [Speeding up Transformer w/ Optimization Strategies](https://www.kaggle.com/rhtsingh/speeding-up-transformer-w-optimization-strategies) notebook explains in-depth 5 optimization strategies with code. All these techniques are promising and can improve the model performance both in terms of speed and accuracy.
  - Dynamic Padding and Uniform Length Batching
  - Gradient Accumulation
  - Freeze Embedding
  - Numeric Precision Reduction
  - Gradient Checkpointing  
   
<font color='#3498DB'><a id="section1"><h2>Preliminary Setup</h2></a></font>

Import all required libraries and import modules/utilities.  
We'll be working with the  in this notebook to visualize various embedding layers.


In [ ]:
import gc; gc.enable()
from IPython.display import clear_output
import pandas as pd
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')

<font color='#3498DB'><a id="section1"><h2>Pooler Output</h2></a></font>

<font color='#3498DB'><a id="section10"><h3>Introduction</h3></a></font>

This is usually the second output from transformer and the simplest one.

Pooler output is the last layer hidden-state of the first token of the sequence (classification token) further processed by a Linear layer and a Tanh activation function. The Linear layer weights are trained from the next sentence prediction (classification) objective during pretraining.

The `last hidden state` is passed into the pooler and this returns the pooled output. We can deactivate pooler outputs by setting `add pooling layer to False` in model config and passing that to model.

<font color='#3498DB'><a id="section10"><h3>Implementation</h3></a></font>

Here we will see how we can utilize the pooler output for downstream task. 

In [ ]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
pooler_output = outputs[1]
logits = nn.Linear(config.hidden_size, 1)(pooler_output) # regression head

print(f'Pooler Output Shape: {pooler_output.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features, outputs
gc.collect();

<font color='#3498DB'><a id="section1"><h2>Last Hidden State Output</h2></a></font>

![last_hidden_state](https://miro.medium.com/max/2120/1*p6PgpOV74U_qLrzr-1_4Zg.png)

<font color='#3498DB'><a id="section10"><h3>Introduction</h3></a></font>
This is the first and default output from models. 

Last Hidden State output is the sequence of hidden-states at the output of the last layer of the model. The output is usually `[batch, maxlen, hidden_state]`, it can be narrowed down to `[batch, 1, hidden_state]` for `[CLS]` token, as the `[CLS]` token is 1st token in the sequence. Here , `[batch, 1, hidden_state]` can be equivalently considered as `[batch, hidden_state]`.

<font color='#3498DB'><a id="section10"><h3>CLS Embeddings</h3></a></font>

![cls](http://www.mccormickml.com/assets/BERT/CLS_token_500x606.png)

<font color='#3498DB'><a id="section10"><h4>Introduction</h4></a></font>
Since Transformers are contextual model, the idea is `[CLS]` token would have captured the entire context and would be sufficient for simple downstream tasks such as classification. Hence, for tasks such as classification using sentence representations, you can use `[batch, hidden_state]`.

<font color='#3498DB'><a id="section10"><h4>Implementation</h4></a></font>
Here we will see how we can utilize the cls embeddings output for downstream task. 

In [ ]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
last_hidden_state = outputs[0]
cls_embeddings = last_hidden_state[:, 0]
logits = nn.Linear(config.hidden_size, 1)(cls_embeddings) # regression head

print(f'Last Hidden State Output Shape: {last_hidden_state.detach().numpy().shape}')
print(f'CLS Embeddings Output Shape: {cls_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features, outputs
gc.collect();

<font color='#3498DB'><a id="section10"><h3>Mean Pooling</h3></a></font>

<font color='#3498DB'><a id="section10"><h4>Introduction</h4></a></font>
We can also consider the last hidden state `[batch, maxlen, hidden_state]`, the average across maxlen dimensions to get averaged/mean embeddings.

There are multiple different ways to do this. We can simply take `torch.mean(last_hidden_state, 1)` but rather we will be implementing something different. We will make use of attention masks as well so that we can ignore padding tokens which is a better way of implementing average embeddings.

<font color='#3498DB'><a id="section10"><h4>Implementation</h4></a></font>

Here we will see how we can utilize the mean pooling embeddings output for downstream task. 

In [ ]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
last_hidden_state = outputs[0]
attention_mask = features['attention_mask']

 - **Step 1:** Expand Attention Mask from `[batch_size, max_len]` to `[batch_size, max_len, hidden_size]`.
 - **Step 2:** Sum Embeddings along `max_len` axis so now we have `[batch_size, hidden_size]`.
 - **Step 3:** Sum Mask along `max_len` axis. This is done so that we can ignore padding tokens.
 - **Step 4:** Take Average.

In [ ]:
input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
sum_mask = input_mask_expanded.sum(1)
sum_mask = torch.clamp(sum_mask, min=1e-9)
mean_embeddings = sum_embeddings / sum_mask
logits = nn.Linear(config.hidden_size, 1)(mean_embeddings) # regression head

print(f'Last Hidden State Output Shape: {last_hidden_state.detach().numpy().shape}')
print(f'Mean Embeddings Output Shape: {mean_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features
gc.collect();

<font color='#3498DB'><a id="section10"><h3>Max Pooling</h3></a></font>

![maxpooling](https://devopedia.org/images/article/241/2120.1575378105.gif)

<font color='#3498DB'><a id="section10"><h4>Introduction</h4></a></font>
Similar to mean pooling we will consider the last hidden state `[batch, maxlen, hidden_state]`, then take max across maxlen dimensions to get max pooling embeddings.

There are multiple different ways to do this as well. We can simply take `torch.max(last_hidden_state, 1)` but rather we will be implementing this similar to mean pooling by utilizing attention masks. 

<font color='#3498DB'><a id="section10"><h4>Implementation</h4></a></font>
Here we will see how we can utilize the max pooling embeddings output for downstream task. 

In [ ]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
last_hidden_state = outputs[0]
attention_mask = features['attention_mask']

In [ ]:
input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
last_hidden_state[input_mask_expanded == 0] = -1e9  # Set padding tokens to large negative value
max_embeddings = torch.max(last_hidden_state, 1)[0]
logits = nn.Linear(config.hidden_size, 1)(max_embeddings) # regression head

print(f'Last Hidden State Output Shape: {last_hidden_state.detach().numpy().shape}')
print(f'Max Embeddings Output Shape: {max_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features
gc.collect();

<font color='#3498DB'><a id="section10"><h3>Mean-Max Pooling (Head)</h3></a></font>

![mean_max](https://media.springernature.com/original/springer-static/image/chp%3A10.1007%2F978-3-030-58323-1_23/MediaObjects/498432_1_En_23_Fig1_HTML.png)

<font color='#3498DB'><a id="section10"><h4>Introduction</h4></a></font>
Mean Max Pooling or Mean Max Head is the most common technique used in many kernels and competitions. We first find mean and max-pooling embeddings, then we concatenate this to have a final representation that is twice the hidden size. The above figure b shows how this is done.

We will be implementing the Kaggle way but one can also find mean and max embeddings like above, then concatenate to have a final resultant vector which will also be the same shape but won't account for padding tokens.

<font color='#3498DB'><a id="section10"><h4>Implementation</h4></a></font>

Here we will see how we can utilize the mean-max pooling embeddings output for downstream task. 

In [ ]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
last_hidden_state = outputs[0]

In [ ]:
mean_pooling_embeddings = torch.mean(last_hidden_state, 1)
_, max_pooling_embeddings = torch.max(last_hidden_state, 1)
mean_max_embeddings = torch.cat((mean_pooling_embeddings, max_pooling_embeddings), 1)
logits = nn.Linear(config.hidden_size*2, 1)(mean_max_embeddings) # twice the hidden size

print(f'Last Hidden State Output Shape: {last_hidden_state.detach().numpy().shape}')
print(f'Mean-Max Embeddings Output Shape: {mean_max_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features
gc.collect();

<font color='#3498DB'><a id="section10"><h3>Conv-1D Pooling</h3></a></font>

<font color='#3498DB'><a id="section10"><h4>Introduction</h4></a></font>

Conv1D pooling is a unique strategy and has been used previously in competitions like Twitter Sentiment Extraction, Jigsaw Unintended Bias in Toxicity etc. Here, we use conv1d layers to get filter unwanted features

Here kernel size 2 means  it considers two tokens for convolution across features. The main idea of conv1d is that it's slides across the tokens and finds the most important tokens by exploring all the 768 features.

<center><img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F1735508%2F208ad2389a980c1d0f6dfd56407d9d23%2F1D-convolutional-example_2x.png?generation=1587806222062877&alt=media" alt="conv1d" style="width:400px;height:500px;"></center>

<font color='#3498DB'><a id="section10"><h4>Implementation</h4></a></font>

Here we will see how we can utilize the Conv1D embeddings output for downstream task. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
last_hidden_state = outputs[0]

In [ ]:
# first define layers
cnn1 = nn.Conv1d(768, 256, kernel_size=2, padding=1)
cnn2 = nn.Conv1d(256, 1, kernel_size=2, padding=1)

last_hidden_state = last_hidden_state.permute(0, 2, 1)
cnn_embeddings = F.relu(cnn1(last_hidden_state))
cnn_embeddings = cnn2(cnn_embeddings)
logits, _ = torch.max(cnn_embeddings, 2)

print(f'Last Hidden State Output Shape: {last_hidden_state.detach().numpy().shape}')
print(f'CNN Embeddings Output Shape: {cnn_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features
gc.collect();

<font color='#3498DB'><a id="section1"><h2>Hidden States Output</h2></a></font>

![weightedpooling](https://miro.medium.com/max/2870/1*8QtJYYJTHcAn0qQEmYsqbw.png)

<font color='#3498DB'><a id="section10"><h3>Introduction</h3></a></font>
This is the third and optional output from transformers. 

Hidden States output has the output of the embeddings + one for the output of each layer of shape `(batch_size, sequence_length, hidden_size)`. Thus, combining hidden-states of the model at the output of each layer plus the initial embedding outputs will give us an output of `(n_layers, batch_size, sequence_length, hidden_size)`. 

There are multiple different ways we can utilize and make use of these embeddings.

<font color='#3498DB'><a id="section10"><h3>CLS Layer Embeddings</h3></a></font>

<font color='#3498DB'><a id="section10"><h4>Introduction</h4></a></font>

Like we saw in the first figure, the BERT authors tried various different layer combinations and in one of them, they use second-to-last layer outputs which performed better than last layer outputs. We have multiple application-dependent strategies for utilizing intermediate representations. 

*Note: To unlock Transformer for giving hidden states as output we need to pass `output_hidden_states` parameter.* 

<font color='#3498DB'><a id="section10"><h4>Implementation</h4></a></font>

Here we will see how we can utilize the Layer-Wise CLS embeddings output for downstream task. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states':True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
all_hidden_states = torch.stack(outputs[2])

layer_index = 11 # second to last hidden layer
cls_embeddings = all_hidden_states[layer_index+1, :, 0] # layer_index+1 as we have 13 layers (embedding + num of blocks)

logits = nn.Linear(config.hidden_size, 1)(cls_embeddings) # regression head

print(f'Hidden States Output Shape: {all_hidden_states.detach().numpy().shape}')
print(f'CLS Embeddings Output Shape: {cls_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features
gc.collect();

<font color='#3498DB'><a id="section10"><h3>Concatenate Pooling</h3></a></font>

<font color='#3498DB'><a id="section10"><h4>Introduction</h4></a></font>

Concatenate Pooling is the technique where we concatenate outputs from different layers into one. In the experiments performed by BERT Authors we saw that Concatenation of Last 4 Layers gave the best results.

<font color='#3498DB'><a id="section10"><h4>Implementation</h4></a></font>

Here we will see how we can utilize the Concatenate Pooling of last 4 Layers output for downstream task. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states':True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
all_hidden_states = torch.stack(outputs[2])

concatenate_pooling = torch.cat(
    (all_hidden_states[-1], all_hidden_states[-2], all_hidden_states[-3], all_hidden_states[-4]),-1
)
concatenate_pooling = concatenate_pooling[:, 0]

logits = nn.Linear(config.hidden_size*4, 1)(concatenate_pooling) # regression head

print(f'Hidden States Output Shape: {all_hidden_states.detach().numpy().shape}')
print(f'Concatenate Pooling Output Shape: {concatenate_pooling.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features
gc.collect();

<font color='#3498DB'><a id="section10"><h3>Weighted Layer Pooling</h3></a></font>

![weighted pooling](https://multithreaded.stitchfix.com/assets/posts/2019-07-07-give-me-jeans/bert.png)

<font color='#3498DB'><a id="section1"><h4>Introduction</h4></a></font>
The output of the last layer may not always be the best representation of the input text during the fine-tuning for downstrea tasks. 

For pre-trained language models, including Transformer, the most transferable contextualized representations of input text tend to occur in the middle layers, while the top layers specialize for language modeling. Therefore, the use of the last layer’s output may restrict the power of the pre-trained representation.

**WeightedLayerPooling** - Token embeddings are the weighted mean of their different hidden layer representations. Weighted Layer Pooling works the best of all pooling techniques be it any given task.

<font color='#3498DB'><a id="section1"><h4>Implementation</h4></a></font>
Here we will see how we can utilize the Weighted Layer Pooling output for downstream task. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states':True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
all_hidden_states = torch.stack(outputs[2])

We have a combined final representation of last four layers. We can now simply take the cls token outputs, concatenate. 
The standard pooling operation as implemented in HuggingFace Transformer for BERT, RoBERTa etc. can also be appled here. Below we simply take the `cls` token outputs and pass it from a Linear layer.

In [ ]:
class WeightedLayerPooling(nn.Module):
    def __init__(self, num_hidden_layers, layer_start: int = 4, layer_weights = None):
        super(WeightedLayerPooling, self).__init__()
        self.layer_start = layer_start
        self.num_hidden_layers = num_hidden_layers
        self.layer_weights = layer_weights if layer_weights is not None \
            else nn.Parameter(
                torch.tensor([1] * (num_hidden_layers+1 - layer_start), dtype=torch.float)
            )

    def forward(self, all_hidden_states):
        all_layer_embedding = all_hidden_states[self.layer_start:, :, :, :]
        weight_factor = self.layer_weights.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand(all_layer_embedding.size())
        weighted_average = (weight_factor*all_layer_embedding).sum(dim=0) / self.layer_weights.sum()
        return weighted_average
    
layer_start = 9
pooler = WeightedLayerPooling(
    config.num_hidden_layers, 
    layer_start=layer_start, layer_weights=None
)
weighted_pooling_embeddings = pooler(all_hidden_states)
weighted_pooling_embeddings = weighted_pooling_embeddings[:, 0]
logits = nn.Linear(config.hidden_size, 1)(weighted_pooling_embeddings)

print(f'Hidden States Output Shape: {all_hidden_states.detach().numpy().shape}')
print(f'Weighted Pooling Output Shape: {weighted_pooling_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features
gc.collect();

<font color='#3498DB'><a id="section10"><h3>LSTM/GRU Pooling</h3></a></font>

![weighted pooling](https://media.arxiv-vanity.com/render-output/4765402/x1.png)

<font color='#3498DB'><a id="section1"><h4>Introduction</h4></a></font>
Representation of the hidden states $h_{CLS}$ is a special sequence: an abstract-tospecific sequence. Since LSTM network is inherently suitable for processing sequential information, we can use a LSTM network to connect all intermediate representations of the [CLS] token, and the output of the last LSTM cell is used as the final representation. Formally,

$$o = h^L_{LSTM} =LSTM(h^i_{CLS}), i ∈ [1, L]$$

<font color='#3498DB'><a id="section1"><h4>Implementation</h4></a></font>
Here we will see how we can utilize the LSTM Pooling output for downstream task. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states':True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
all_hidden_states = torch.stack(outputs[2])

In [ ]:
class LSTMPooling(nn.Module):
    def __init__(self, num_layers, hidden_size, hiddendim_lstm):
        super(LSTMPooling, self).__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size = hidden_size
        self.hiddendim_lstm = hiddendim_lstm
        self.lstm = nn.LSTM(self.hidden_size, self.hiddendim_lstm, batch_first=True)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, all_hidden_states):
        ## forward
        hidden_states = torch.stack([all_hidden_states[layer_i][:, 0].squeeze()
                                     for layer_i in range(1, self.num_hidden_layers+1)], dim=-1)
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)
        out, _ = self.lstm(hidden_states, None)
        out = self.dropout(out[:, -1, :])
        return out

hiddendim_lstm = 256
pooler = LSTMPooling(config.num_hidden_layers, config.hidden_size, hiddendim_lstm)
lstm_pooling_embeddings = pooler(all_hidden_states)
logits = nn.Linear(hiddendim_lstm, 1)(lstm_pooling_embeddings) # regression head

print(f'Hidden States Output Shape: {all_hidden_states.detach().numpy().shape}')
print(f'LSTM Pooling Output Shape: {lstm_pooling_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features
gc.collect();

<font color='#3498DB'><a id="section10"><h3>Attention Pooling</h3></a></font>

<font color='#3498DB'><a id="section1"><h4>Introduction</h4></a></font>
Intuitively, attention operation can learn the contribution of each $h^i_{CLS}$. We can use a dot-product attention module to dynamically combine all intermediates:
$$o = W^T_{h} softmax(qh^T_{CLS})h_{CLS}$$

where $W^T_{h}$ and $q$ are learnable weights.
Finally, we pass the pooled output o to a fullyconnected layer for label prediction:
$$y = sof tmax(W^T_{o}o + bo)$$

<font color='#3498DB'><a id="section1"><h4>Implementation</h4></a></font>
Here we will see how we can utilize the Attention Pooling output for downstream task. 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states':True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
all_hidden_states = torch.stack(outputs[2])

In [ ]:
class AttentionPooling(nn.Module):
    def __init__(self, num_layers, hidden_size, hiddendim_fc):
        super(AttentionPooling, self).__init__()
        self.num_hidden_layers = num_layers
        self.hidden_size = hidden_size
        self.hiddendim_fc = hiddendim_fc
        self.dropout = nn.Dropout(0.1)

        q_t = np.random.normal(loc=0.0, scale=0.1, size=(1, self.hidden_size))
        self.q = nn.Parameter(torch.from_numpy(q_t)).float()
        w_ht = np.random.normal(loc=0.0, scale=0.1, size=(self.hidden_size, self.hiddendim_fc))
        self.w_h = nn.Parameter(torch.from_numpy(w_ht)).float()

    def forward(self, all_hidden_states):
        hidden_states = torch.stack([all_hidden_states[layer_i][:, 0].squeeze()
                                     for layer_i in range(1, self.num_hidden_layers+1)], dim=-1)
        hidden_states = hidden_states.view(-1, self.num_hidden_layers, self.hidden_size)
        out = self.attention(hidden_states)
        out = self.dropout(out)
        return out

    def attention(self, h):
        v = torch.matmul(self.q, h.transpose(-2, -1)).squeeze(1)
        v = F.softmax(v, -1)
        v_temp = torch.matmul(v.unsqueeze(1), h).transpose(-2, -1)
        v = torch.matmul(self.w_h.transpose(1, 0), v_temp).squeeze(2)
        return v

hiddendim_fc = 128
pooler = AttentionPooling(config.num_hidden_layers, config.hidden_size, hiddendim_fc)
attention_pooling_embeddings = pooler(all_hidden_states)
logits = nn.Linear(hiddendim_fc, 1)(attention_pooling_embeddings) # regression head

print(f'Hidden States Output Shape: {all_hidden_states.detach().numpy().shape}')
print(f'Attention Pooling Output Shape: {attention_pooling_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features
gc.collect();

<font color='#3498DB'><a id="section10"><h3>WKPooling</h3></a></font>

![WKPooling](https://raw.githubusercontent.com/BinWang28/BERT_Sentence_Embedding/master/figure1.png)

<font color='#3498DB'><a id="section1"><h4>Introduction</h4></a></font>
Pooling based on the paper: "SBERT-WK: A Sentence Embedding Method ByDissecting BERT-based Word Models".

It consists of the following
two steps:
 - Determine a unified word representation for each word in a sentence by integrating its representations across layers by examining its alignment and novelty properties.  
 - Conduct a weighted average of unified word representations based on the word importance measure to yield the ultimate sentence embedding vector.

You can find more details in the paper.

<font color='#3498DB'><a id="section1"><h4>Implementation</h4></a></font>
Here we will see how we can utilize the WK Pooling output for downstream task. 

*Note: SBERT-WK uses QR decomposition. torch QR decomposition is currently extremely slow when run on GPU. Hence, the tensor is first transferred to the CPU before it is applied. This makes this pooling method rather slow.*

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)
logging.set_verbosity_error()
logging.set_verbosity_warning()

train_text = train['excerpt'][:16].tolist()
max_seq_length = 256
_pretrained_model = 'roberta-base'

config = AutoConfig.from_pretrained(_pretrained_model)
config.update({'output_hidden_states':True})
model = AutoModel.from_pretrained(_pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(_pretrained_model)

clear_output()

features = tokenizer.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

with torch.no_grad():
    outputs = model(features['input_ids'], features['attention_mask'])
all_hidden_states = torch.stack(outputs[2])

In [ ]:
class WKPooling(nn.Module):
    def __init__(self, layer_start: int = 4, context_window_size: int = 2):
        super(WKPooling, self).__init__()
        self.layer_start = layer_start
        self.context_window_size = context_window_size

    def forward(self, all_hidden_states):
        ft_all_layers = all_hidden_states
        org_device = ft_all_layers.device
        all_layer_embedding = ft_all_layers.transpose(1,0)
        all_layer_embedding = all_layer_embedding[:, self.layer_start:, :, :]  # Start from 4th layers output

        # torch.qr is slow on GPU (see https://github.com/pytorch/pytorch/issues/22573). So compute it on CPU until issue is fixed
        all_layer_embedding = all_layer_embedding.cpu()

        attention_mask = features['attention_mask'].cpu().numpy()
        unmask_num = np.array([sum(mask) for mask in attention_mask]) - 1  # Not considering the last item
        embedding = []

        # One sentence at a time
        for sent_index in range(len(unmask_num)):
            sentence_feature = all_layer_embedding[sent_index, :, :unmask_num[sent_index], :]
            one_sentence_embedding = []
            # Process each token
            for token_index in range(sentence_feature.shape[1]):
                token_feature = sentence_feature[:, token_index, :]
                # 'Unified Word Representation'
                token_embedding = self.unify_token(token_feature)
                one_sentence_embedding.append(token_embedding)

            ##features.update({'sentence_embedding': features['cls_token_embeddings']})

            one_sentence_embedding = torch.stack(one_sentence_embedding)
            sentence_embedding = self.unify_sentence(sentence_feature, one_sentence_embedding)
            embedding.append(sentence_embedding)

        output_vector = torch.stack(embedding).to(org_device)
        return output_vector

    def unify_token(self, token_feature):
        ## Unify Token Representation
        window_size = self.context_window_size

        alpha_alignment = torch.zeros(token_feature.size()[0], device=token_feature.device)
        alpha_novelty = torch.zeros(token_feature.size()[0], device=token_feature.device)

        for k in range(token_feature.size()[0]):
            left_window = token_feature[k - window_size:k, :]
            right_window = token_feature[k + 1:k + window_size + 1, :]
            window_matrix = torch.cat([left_window, right_window, token_feature[k, :][None, :]])
            Q, R = torch.qr(window_matrix.T)

            r = R[:, -1]
            alpha_alignment[k] = torch.mean(self.norm_vector(R[:-1, :-1], dim=0), dim=1).matmul(R[:-1, -1]) / torch.norm(r[:-1])
            alpha_alignment[k] = 1 / (alpha_alignment[k] * window_matrix.size()[0] * 2)
            alpha_novelty[k] = torch.abs(r[-1]) / torch.norm(r)

        # Sum Norm
        alpha_alignment = alpha_alignment / torch.sum(alpha_alignment)  # Normalization Choice
        alpha_novelty = alpha_novelty / torch.sum(alpha_novelty)

        alpha = alpha_novelty + alpha_alignment
        alpha = alpha / torch.sum(alpha)  # Normalize

        out_embedding = torch.mv(token_feature.t(), alpha)
        return out_embedding

    def norm_vector(self, vec, p=2, dim=0):
        ## Implements the normalize() function from sklearn
        vec_norm = torch.norm(vec, p=p, dim=dim)
        return vec.div(vec_norm.expand_as(vec))

    def unify_sentence(self, sentence_feature, one_sentence_embedding):
        ## Unify Sentence By Token Importance
        sent_len = one_sentence_embedding.size()[0]

        var_token = torch.zeros(sent_len, device=one_sentence_embedding.device)
        for token_index in range(sent_len):
            token_feature = sentence_feature[:, token_index, :]
            sim_map = self.cosine_similarity_torch(token_feature)
            var_token[token_index] = torch.var(sim_map.diagonal(-1))

        var_token = var_token / torch.sum(var_token)
        sentence_embedding = torch.mv(one_sentence_embedding.t(), var_token)

        return sentence_embedding
    
    def cosine_similarity_torch(self, x1, x2=None, eps=1e-8):
        x2 = x1 if x2 is None else x2
        w1 = x1.norm(p=2, dim=1, keepdim=True)
        w2 = w1 if x2 is x1 else x2.norm(p=2, dim=1, keepdim=True)
        return torch.mm(x1, x2.t()) / (w1 * w2.t()).clamp(min=eps)

In [ ]:
pooler = WKPooling(layer_start=9)
wkpooling_embeddings = pooler(all_hidden_states)
logits = nn.Linear(config.hidden_size, 1)(wkpooling_embeddings) # regression head

print(f'Hidden States Output Shape: {all_hidden_states.detach().numpy().shape}')
print(f'WKPooling Output Shape: {wkpooling_embeddings.detach().numpy().shape}')
print(f'Logits Shape: {logits.detach().numpy().shape}')

del config, model, tokenizer, features
gc.collect();

<font color='#3498DB'><a id="section1"><h2>Review</h2></a></font>

Going back to figure 1, we can see that we have implemented all of the techniques that the BERT authors used to investigate and many more. One, can easily plug and play any of the above head / pooler into their model and finetune on a downstream task.

![embedding_layers](http://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png)

<font color='#3498DB'><a id="section1"><h2>Case Study</h2></a></font>
Han Xiao created an open-source project named bert-as-service on GitHub which is intended to create word embeddings for your text using BERT `bert-as-service`, by default, uses the outputs from the second-to-last layer of the model.

His observations are - 
 - The embeddings start in the first layer as having no contextual information.
 - As the embeddings move deeper into the network, they pick up more and more contextual information with each layer.
 - As you approach the final layer, however, you start picking up information that is specific to BERT’s pre-training tasks (the “Masked Language Model” (MLM) and “Next Sentence Prediction” (NSP)).
    - What we want are embeddings that encode the word meaning well…
    - BERT is motivated to do this, but it is also motivated to encode anything else that would help it determine what a missing word is (MLM), or whether the second sentence came after the first (NSP).
 - The second-to-last layer is what Han settled on as a reasonable sweet spot.
 
<font color='#3498DB'><a id="section112"><h2>References and Resources</h2></a></font>
 
Here I would like to acknowledge all the publications, blogs, notebooks etc. without which making this kernel would not have been possible, 

 - **Papers:**
   - [Deepening Hidden Representations from Pre-trained Language Models](https://arxiv.org/pdf/1911.01940v2.pdf)
   - [Linguistic Knowledge and Transferability of Contextual Representations](https://www.aclweb.org/anthology/N19-1112.pdf)
   - [What does BERT learn about the structure of language?](https://www.aclweb.org/anthology/P19-1356.pdf)
   - [Dissecting Contextual Word Embeddings: Architecture and Representation](https://www.aclweb.org/anthology/D18-1179.pdf)
   - [SDNET: CONTEXTUALIZED ATTENTION-BASED DEEP NETWORK FOR CONVERSATIONAL QUESTION ANSWERING](https://arxiv.org/pdf/1812.03593.pdf)
   - [Utilizing BERT Intermediate Layers for Aspect Based Sentiment Analysis
and Natural Language Inference](https://arxiv.org/pdf/2002.04815.pdf)
   - [WHAT DO YOU LEARN FROM CONTEXT? PROBING FOR SENTENCE STRUCTURE IN CONTEXTUALIZED WORD REPRESENTATIONS](https://arxiv.org/pdf/1905.06316.pdf)
   - [SBERT-WK: A Sentence Embedding Method by Dissecting BERT-based Word Models](https://arxiv.org/pdf/2002.06652.pdf)
 
 - **Blogs**
   - [BERT Word Embeddings Tutorial](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#4-appendix)
   - [Visualize BERT sequence embeddings: An unseen way](https://towardsdatascience.com/visualize-bert-sequence-embeddings-an-unseen-way-1d6a351e4568)
   - [Deconstructing BERT, Part 2: Visualizing the Inner Workings of Attention](https://towardsdatascience.com/deconstructing-bert-part-2-visualizing-the-inner-workings-of-attention-60a16d86b5c1)
   - [Writing Math Equations in Jupyter Notebook: A Naive Introduction](https://medium.com/analytics-vidhya/writing-math-equations-in-jupyter-notebook-a-naive-introduction-a5ce87b9a214)

 - **GitHub**
   - [Sentence Transformers: Multilingual Sentence, Paragraph, and Image Embeddings using BERT & Co.](https://github.com/UKPLab/sentence-transformers)
   - [FLAIR](https://github.com/flairNLP/flair)
   - [BERT Fine-tuning for Aspect Based Sentiment Analysis](https://github.com/avinashsai/BERT-Aspect)
   - [Interpreting Bidirectional Encoder Representations from Transformers](https://github.com/ganeshjawahar/interpret_bert)
   - [BertViz](https://github.com/jessevig/bertviz)
 
 - **Kaggle Kernels and Discussion**
   - [Jigsaw Unintended Bias in Toxicity - 1st Place](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/discussion/103280)
   - [Jigsaw Unintended Bias in Toxicity - 4th Place](https://www.kaggle.com/iezepov/wombat-inference-kernel)
   - [Jigsaw Unintended Bias in Toxicity - 8th Place](https://www.kaggle.com/haqishen/jigsaw-predict)
   - [Twitter Sentiment Extraction - 2nd Place](https://www.kaggle.com/hiromoon166/inference-8models-seed100101-bucketing-2-ver2)
   - [Twitter Sentiment Extraction - 3rd Place](https://www.kaggle.com/suicaokhoailang/final-sub-tweet)
   - [Twitter Sentiment Extraction - 7th Place](https://www.kaggle.com/naivelamb/roberta-base-ensemble)
   - [Twitter Sentiment Extraction - TensorFlow roBERTa - [0.712]](https://www.kaggle.com/al0kharba/tensorflow-roberta-0-712)
   - [Jigsaw Multilingual Toxic Comment Classification - 4th Place](https://www.kaggle.com/c/jigsaw-multilingual-toxic-comment-classification/discussion/160980)
   - [CommonLit Readability Prize - Step 1: Create Folds](https://www.kaggle.com/abhishek/step-1-create-folds)
 
<font color='#3498DB'><a id="section6"><h2>Ending Notes</h2></a></font>

- There are many other strategies which I haven't covererd and which one do further research on,
    - Dense Pooling
    - Word Weight (TF-IDF) Pooling
    - Async Pooling
    - Parallel / Heirarchical Aggregation
    
- The performance of pooling strategies will vary from task to task and there is no one technique that always performs the best. 

- I prefer to use WeightLayerPooling of last 4 layers.

- More comprehensive repository for learning and implementing Transformers for various tasks can be found [here](https://notebooks.quantumstat.com/), [here](https://huggingface.co/transformers/master/community.html#community-notebooks) and [here](https://huggingface.co/transformers/notebooks.html)  

<font color='#3498DB'><a id="section2"><h2>Thanks & Please Do Upvote!</h2></a></font>